In [5]:
from spacy.en import English
parser = English()

In [7]:
import foodbornenyc.models.models as models
from foodbornenyc.models.businesses import Business, business_category_table
from foodbornenyc.models.documents import YelpReview, Tweet, Document
from foodbornenyc.models.locations import Location
from foodbornenyc.models.metadata import metadata

In [34]:
import xlrd
f = xlrd.open_workbook('data/yelp_sick_classifier_data.xlsx')
sheet1 = f.sheet_by_index(0)

In [102]:
from sklearn.externals import joblib
from foodbornenyc.settings import yelp_classify_config as config

sick = joblib.load("../"+config['model_file'])
sick

Pipeline(steps=[('count', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.9, max_features=None, min_df=2,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
    ...ti_class='ovr', penalty='l2', random_state=0,
          solver='liblinear', tol=0.0001, verbose=0))])

In [94]:
reviews = []
for i, (rev, label) in enumerate(zip(sheet1.col(1), sheet1.col(2))):
    if i == 0: continue
    reviews.append({"text":rev.value, "label":label.value})
print reviews

[{'text': u'My friends and I ordered 3 burgers and a pitcher of sangria. Everything tasted good. Later on in the day we all got a stomach ache then 1 by 1 we all had to use the restroom several times before the days end. I love this place, but something there made us sick. \n\nIf I go back, I&#39;m going to stick with bottled beer.', 'label': 1.0}, {'text': u'Went here yesterday with a friend.  We both got food poisoning, most likely from the cajun chicken. Resulted in a pretty rough night last night and for my friend, a rough day today (will spare the details, but you can imagine).\n\nWould recommend steering clear unless you&#39;re looking to ruin about 24 hours of your life.\n\nGood happy hour special though after the last 24 hours I&#39;d stick to bottled beverages, trust factor on anything prepared there is low.\n\nAs an aside, during a previous trip, we were required to order at our tables rather than the counter, this caused the length of our visit to nearly double since they we

In [95]:
textonly = [a["text"] for a in reviews]
predictions = sick.predict_proba(textonly)

In [96]:
predictions

array([[ 0.31401907,  0.68598093],
       [ 0.27546785,  0.72453215],
       [ 0.3627709 ,  0.6372291 ],
       ..., 
       [ 0.16799656,  0.83200344],
       [ 0.26726342,  0.73273658],
       [ 0.58887452,  0.41112548]])

In [101]:
# determine true/false positive/negative rates
tp_rate = 0.0
fp_rate = 0.0
tn_rate = 0.0
fn_rate = 0.0

for review, pred in zip(reviews, predictions):
    if review['label'] == 1.0 and pred[0] > 0.5: tp_rate += 1
    elif review['label'] == 1.0 and pred[0] < 0.5: fn_rate += 1
    elif review['label'] == 0.0 and pred[0] > 0.5: fp_rate += 1
    elif review['label'] == 0.0 and pred[0] < 0.5: tn_rate += 1
tp_rate /= len(reviews)
fn_rate /= len(reviews)
fp_rate /= len(reviews)
tn_rate /= len(reviews)
print tp_rate, fn_rate, fp_rate, tn_rate

0.048132183908 0.484913793103 0.29525862069 0.171695402299


In [98]:
# determine roc score
import numpy as np
from sklearn.metrics import roc_auc_score
label = np.array([review['label'] for review in reviews])
pred = np.array([pred[0] for pred in predictions])
roc_auc_score(label, pred, average='micro')

0.12802612481857764

In [105]:
sheet2 = xlrd.open_workbook('data/sick_test_preds.xlsx').sheet_by_index(0)
reviews2 = []
for i, (rev, label) in enumerate(zip(sheet2.col(0), sheet2.col(3))):
    if i == 0: continue
    reviews2.append({"text":rev.value, "label":label.value})
print reviews2

[{'text': u'"Bad service from the start, everything was horrible, worst Middles Eastern -Lebanese dishes I have ever had.  Everything was cold and old- we really felt sick after we left.\n\nWe were not given a menu for ten minutes into sitting down.\n\nFrom appetizers to entrees to desert each time we ordered we literally waited 20 minutes each time. Not to mention everything tasted like crap!\nI have never had such a bad service, it was truly terrible.  Chicken and Rice on the street tastes 100 times better than their food.  \n\nThe owner is cheap and extremely rude.\n\nThere is no way in the world anyone could have had a + experience going here.  I am sure these positive stars by others are fake and made up.\n\nDo not waste your time and money by going here!\nWill never ever go there again!"', 'label': 1.0}, {'text': u'"CONS:\n*Revolting food -- and I do mean that in a literal sense. It gave me food poisoning. I\'m not sure how food that greasy was also so bland. I would have expecte

In [108]:
textonly = [a["text"] for a in reviews2]
predictions = sick.predict_proba(textonly)
predictions

array([[ 0.32358796,  0.67641204],
       [ 0.37314294,  0.62685706],
       [ 0.26726342,  0.73273658],
       [ 0.17682572,  0.82317428],
       [ 0.26320034,  0.73679966],
       [ 0.3630461 ,  0.6369539 ],
       [ 0.47305474,  0.52694526],
       [ 0.49552092,  0.50447908],
       [ 0.67699534,  0.32300466],
       [ 0.60969204,  0.39030796],
       [ 0.73491791,  0.26508209],
       [ 0.45593356,  0.54406644],
       [ 0.33782023,  0.66217977],
       [ 0.19902779,  0.80097221],
       [ 0.35853315,  0.64146685],
       [ 0.34597509,  0.65402491],
       [ 0.60633742,  0.39366258],
       [ 0.33404204,  0.66595796],
       [ 0.47114027,  0.52885973],
       [ 0.25178498,  0.74821502],
       [ 0.25453609,  0.74546391],
       [ 0.36629017,  0.63370983],
       [ 0.37797976,  0.62202024],
       [ 0.63110973,  0.36889027],
       [ 0.38126045,  0.61873955],
       [ 0.30931569,  0.69068431],
       [ 0.3746919 ,  0.6253081 ],
       [ 0.55224104,  0.44775896],
       [ 0.3757524 ,

In [107]:
label = np.array([review['label'] for review in reviews2])
pred = np.array([pred[0] for pred in predictions])
roc_auc_score(label, pred, average='micro')

0.13236964377903976

In [ ]:
# building a new classifier